In [1]:
import pandas as pd
import os

In [2]:
train = pd.read_csv(os.path.join('train.csv'))
val = pd.read_csv(os.path.join('validation.csv'))
test = pd.read_csv(os.path.join('test.csv'))

In [3]:
xgboost_pred = pd.read_csv(os.path.join('xgboost_val.csv'))
lr_pred = pd.read_csv(os.path.join('lr_val.csv'))

In [12]:
val_pred = xgboost_pred['click'] * 0.85 + lr_pred['click'] * 0.15

In [13]:
sum([round(x) for x in val_pred]) / len(val)

0.00017767541334210742

In [14]:
avgCTR = sum(train['click']) / len(train)
print(avgCTR)

0.0007375623256619447


In [15]:
def compare_performance(metrics_list, best_metrics):
    return metrics_list[0] >= best_metrics[0] and metrics_list[1] >= best_metrics[1]

In [16]:
new_val = val.copy()
results = []

for base_bid in range(3, 100):
    bidprices = [x * base_bid / avgCTR for x in val_pred]
    new_val['bidprice'] = bidprices
    new_val.loc[new_val.advertiser == 2259, 'bidprice'] = 15
    new_val.loc[new_val.advertiser == 2261, 'bidprice'] = 15   
    budget = 6250
    suc_bids = new_val.query('bidprice >= payprice and bidprice >= slotprice')
    cost = 0
    clicks = 0
    imps = 0
    i = 0
    for index, row in suc_bids.iterrows():
        if cost <= budget:
            cost += row['payprice'] / 1000
            clicks += row['click']
            imps += 1
            
    eCPC = cost / clicks if clicks > 0 else float('inf')
    metrics_list = [clicks, clicks / imps * 100, cost, cost / imps * 1000, eCPC]
    results.append([base_bid, clicks, clicks / imps * 100, cost, cost / imps * 1000, eCPC])
    
results = pd.DataFrame(results)
results = results.rename(columns={0: "base_bid", 1: "clicks",
    2: "CTR", 3: "cost", 4: "avg CPM", 5: "eCPC"})
results.to_csv(os.path.join('linear_bid_xgboost_lr.csv'), index=False)

In [17]:
results.loc[results.clicks == max(results['clicks'])]

,base_bid,clicks,CTR,cost,avg CPM,eCPC
66,69,163,0.126653,6250.026,48.563505,38.343718
